In [ ]:
!pip install image-similarity-measures

In [7]:
import os
import cv2
import pandas as pd
import numpy as np
import image_similarity_measures
from sys import argv
from image_similarity_measures.quality_metrics import *


In [2]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/working')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df_ML_based = pd.read_csv('results.csv')
df_ML_based

,TestImage,QueryImage,Match_No_Match,Similarity_Score
0,test1,Cattle1_1,Match,0.216
1,test1,Cattle1_2,Match,0.171
2,test1,Cattle1_3,Match,0.273
3,test1,Cattle2_1,Match,0.140
4,test1,Cattle2_2,Match,0.263
5,test1,Cattle2_3,Match,0.173
6,test2,Cattle1_1,Match,0.248
7,test2,Cattle1_2,Match,0.219
8,test2,Cattle1_3,No_Match,0.351
9,test2,Cattle2_1,Match,0.267


**Preprocessing**

In [ ]:
#This method will crop image from original image so that it becomes better suited to check pattern match

IMG_SIZE = 400
q_lst = os.listdir('cattles')
t_lst = os.listdir('test')


def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

def circle_crop(img, sigmaX = 10):   
    """
    Create circular crop around image centre    
    """    
    img = crop_image_from_gray(img)    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img=cv2.addWeighted(img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
    return img 

def preprocess_image(file):
    input_filepath = os.path.join('./','cattles','{}'.format(file))
    output_filepath = os.path.join('./','cattles_resized','{}'.format(file))
    
    img = cv2.imread(input_filepath)
    img = circle_crop(img) 
    cv2.imwrite(output_filepath, cv2.resize(img, (IMG_SIZE,IMG_SIZE)))

'''This Function uses Multi processing for faster saving of images into folder'''

def multiprocess_image_processor(process:int, imgs:list):
    """
    Inputs:
        process: (int) number of process to run
        imgs:(list) list of images
    """
    print(f'MESSAGE: Running {process} process')
    results = ThreadPool(process).map(preprocess_image, imgs)
    return results



multiprocess_image_processor(2, q_lst) #Google Colab has 2 CPUs
multiprocess_image_processor(2, t_lst)

In [9]:
lst_results = []
ssim_measures = {}
rmse_measures = {}
sre_measures = {}
sam_measures = {}
fsim_measures = {}
issm_measures = {}
psnr_measures = {}
uiq_measures = {}



sample_img = cv2.imread('/content/drive/MyDrive/working/cattles_resized/Cattle1_2.jpg')

scale_percent = 100 # percent of original img size
width = int(sample_img.shape[1] * scale_percent / 100)
height = int(sample_img.shape[0] * scale_percent / 100)
dim = (width, height)

query_dir = 'cattles_resized'
test_dir = 'test_resized'
test_list = sorted(os.listdir(test_dir))
query_list = sorted(os.listdir(query_dir))
#print(lst)

for test_image in test_list:
  test_img_path = os.path.join(test_dir, test_image)
  test_img = cv2.imread(test_img_path)
  #print(test_img.shape)
  for file in query_list:
    #print(file)
    img_path = os.path.join(query_dir, file)
    data_img = cv2.imread(img_path)
    #data_img = cv2.imread(file)
    resized_img = cv2.resize(data_img, dim, interpolation = cv2.INTER_AREA)
    #print(resized_img.shape)
    ssim_measures[file]= ssim(test_img, resized_img)
    rmse_measures[file]= rmse(test_img, resized_img)
    sre_measures[file]= sre(test_img, resized_img)
    sam_measures[file]= ssim(test_img, resized_img)
    fsim_measures[file]= ssim(test_img, resized_img)
    issm_measures[file]= ssim(test_img, resized_img)
    psnr_measures[file]= ssim(test_img, resized_img)
    uiq_measures[file]= ssim(test_img, resized_img)
  
  
  df_result = pd.DataFrame.from_dict(ssim_measures, orient='index', columns=['Similarity_Score'])
  df_result['TestImage'] = test_image
  lst_results.append(df_result)

df_results = pd.concat(lst_results)
df_results.index = df_results.index.rename('QueryImage')
df_results = df_results.reindex(['TestImage','Similarity_Score'], axis=1)
df_results

,TestImage,Similarity_Score
QueryImage,,
Cattle1_1.jpg,test1.jpg,0.736684
Cattle1_2.jpg,test1.jpg,0.788232
Cattle1_3.jpg,test1.jpg,0.737421
Cattle2_1.jpg,test1.jpg,0.813039
Cattle2_2.jpg,test1.jpg,0.802955
Cattle2_3.jpg,test1.jpg,0.812167
Cattle1_1.jpg,test2.jpg,0.711573
Cattle1_2.jpg,test2.jpg,0.751512
Cattle1_3.jpg,test2.jpg,0.726364


If Similarity_Score is less than a threshold value then it's 'no match' otherwise it's match.

In [17]:
max_value = df_results['Similarity_Score'].max()
min_value = df_results['Similarity_Score'].min()
diff = max_value - min_value

threshold = min_value + 0.5 * diff # here we can put according to problem in place of 0.8
threshold

0.7957338835610783

In [18]:
df_results['match_no_match'] = df_results['Similarity_Score'] - threshold
df_results['match_no_match'] = df_results['match_no_match'].apply(lambda x : 'Yes' if x > 0 else 'No')
df_results

,TestImage,Similarity_Score,match_no_match
QueryImage,,,
Cattle1_1.jpg,test1.jpg,0.736684,No
Cattle1_2.jpg,test1.jpg,0.788232,No
Cattle1_3.jpg,test1.jpg,0.737421,No
Cattle2_1.jpg,test1.jpg,0.813039,Yes
Cattle2_2.jpg,test1.jpg,0.802955,Yes
Cattle2_3.jpg,test1.jpg,0.812167,Yes
Cattle1_1.jpg,test2.jpg,0.711573,No
Cattle1_2.jpg,test2.jpg,0.751512,No
Cattle1_3.jpg,test2.jpg,0.726364,No
